In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn import ensemble    
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import lightgbm as lgb

In [2]:
training = pd.read_csv('/home/shin/Documents/Data_mining/train.csv', sep = ',')

In [3]:
data = pd.read_csv('/home/shin/Documents/Data_mining/train.csv', sep = ',')

In [4]:
challenge = pd.read_csv('/home/shin/Documents/Data_mining/challenge.csv', sep = ',')

In [5]:
data = pd.concat([data, challenge], ignore_index=True)

In [6]:
## Parsing Date and time (Seconds omited because they are all '0')
import datetime
date = data.Datum
Year = []
Month = []
Day = []
Hour = []
Minute = []
for i in date:
    dateparse = datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S")
    Year.append(dateparse.year)
    Month.append(dateparse.month)
    Day.append(dateparse.day)
    Hour.append(dateparse.hour)
    Minute.append(dateparse.minute)
data.insert(0, 'Year', Year)
data.insert(1, 'Month', Month)
data.insert(2, 'Day', Day)
data.insert(3, 'Hour', Hour)
data.insert(4, 'Min', Minute)
data = data.drop(['Datum'], axis = 1)

date_challenge = challenge.Datum
Year_challenge = []
Month_challenge = []
Day_challenge = []
Hour_challenge = []
Minute_challenge = []
for i in date_challenge:
    dateparse_challenge = datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S")
    Year_challenge.append(dateparse_challenge.year)
    Month_challenge.append(dateparse_challenge.month)
    Day_challenge.append(dateparse_challenge.day)
    Hour_challenge.append(dateparse_challenge.hour)
    Minute_challenge.append(dateparse_challenge.minute)
challenge.insert(0, 'Year', Year_challenge)
challenge.insert(1, 'Month', Month_challenge)
challenge.insert(2, 'Day', Day_challenge)
challenge.insert(3, 'Hour', Hour_challenge)
challenge.insert(4, 'Min', Minute_challenge)
challenge = challenge.drop(['Datum'], axis = 1)

In [7]:
data.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Min', 'Windgeschwindigkeit48M',
       'Windgeschwindigkeit100M', 'Windgeschwindigkeit152M', 'Windrichtung48M',
       'Windrichtung100M', 'Windrichtung152M', 'Windgeschwindigkeit100MP10',
       'Windgeschwindigkeit100MP20', 'Windgeschwindigkeit100MP30',
       'Windgeschwindigkeit100MP40', 'Windgeschwindigkeit100MP50',
       'Windgeschwindigkeit100MP60', 'Windgeschwindigkeit100MP70',
       'Windgeschwindigkeit100MP80', 'Windgeschwindigkeit100MP90',
       'Interpoliert', 'Verfügbare_Kapazität', 'Output'],
      dtype='object')

In [8]:
# Does not improve the result
# ratio = data.Output / data.Verfügbare_Kapazität
# Windgeschwindigkeit48M_tanh = np.tanh(data.Windgeschwindigkeit48M)
# Windgeschwindigkeit48M_log = np.log(data.Windgeschwindigkeit48M)
# Windgeschwindigkeit100M_log = np.log(data.Windgeschwindigkeit100M)
# Windgeschwindigkeit152M_log = np.log(data.Windgeschwindigkeit152M)
# windspeed_sq = np.sqrt(windspeed_avg)
# windspeed_log = np.log(windspeed_avg) 
# data.insert(11, 'windspeed_log', windspeed_log)
# windspeed_tan = np.tanh(windspeed_avg)
# windspeed_max = windspeed.max(axis = 1)
# windspeedmp = data.loc[:len(data), 'Windgeschwindigkeit100MP10':'Windgeschwindigkeit100MP90']
# windspeedmp_avg = windspeedmp.mean(axis = 1)
# windspeedmp_sum = windspeedmp.sum(axis = 1)
# windspeedmp_max = windspeedmp.max(axis = 1)
# Windgeschwindigkeit100MP15 = data.loc[:len(data), 'Windgeschwindigkeit100MP10':'Windgeschwindigkeit100MP20'].mean(axis = 1)
# Windgeschwindigkeit100MP25 = data.loc[:len(data), 'Windgeschwindigkeit100MP20':'Windgeschwindigkeit100MP30'].mean(axis = 1)
# winddirection = data.loc[:len(data), 'Windrichtung48M':'Windrichtung152M'].sum(axis = 1)

In [9]:
'''
season = []
for i in range(len(data)):
    if data.Month[i] == 4 or 5 or 6 or 7 or 8 or 9 or 10:
        season.append(0)
    else:
        season.append(1)
data.insert(11, 'season', season)
'''

"\nseason = []\nfor i in range(len(data)):\n    if data.Month[i] == 4 or 5 or 6 or 7 or 8 or 9 or 10:\n        season.append(0)\n    else:\n        season.append(1)\ndata.insert(11, 'season', season)\n"

In [10]:
# Feature Engineering 
# Windspeed average / SUM / ^2 ... (Coz they actually improved the result)
windspeed = data.loc[:len(data), 'Windgeschwindigkeit48M':'Windgeschwindigkeit152M']
windspeed_avg = windspeed.mean(axis = 1)
windspeed_sum = windspeed.sum(axis = 1)
Windgeschwindigkeit48M_2 = np.power(data.Windgeschwindigkeit48M, 2)
Windgeschwindigkeit100M_2 = np.power(data.Windgeschwindigkeit100M, 2)
Windgeschwindigkeit152M_2 = np.power(data.Windgeschwindigkeit152M, 2)
data.insert(8, 'windspeed_avg', windspeed_avg)
data.insert(9, 'windspeed_sum', windspeed_sum)
data.insert(10, 'Windgeschwindigkeit48M_2', Windgeschwindigkeit48M_2)
data.insert(11, 'Windgeschwindigkeit100M_2', Windgeschwindigkeit100M_2)
data.insert(12, 'Windgeschwindigkeit152M_2', Windgeschwindigkeit152M_2)
data = data.drop(['Windgeschwindigkeit100MP10','Windgeschwindigkeit100MP20','Windgeschwindigkeit100MP30','Windgeschwindigkeit100MP40','Windgeschwindigkeit100MP50','Windgeschwindigkeit100MP60','Windgeschwindigkeit100MP70','Windgeschwindigkeit100MP80','Windgeschwindigkeit100MP90'], axis = 1)

In [11]:
ch_windspeed = challenge.loc[:len(challenge), 'Windgeschwindigkeit48M':'Windgeschwindigkeit152M']
ch_windspeed_avg = ch_windspeed.mean(axis = 1)
ch_windspeed_sum = ch_windspeed.sum(axis = 1)
ch_Windgeschwindigkeit48M_2 = np.power(challenge.Windgeschwindigkeit48M, 2)
ch_Windgeschwindigkeit100M_2 = np.power(challenge.Windgeschwindigkeit100M, 2)
ch_Windgeschwindigkeit152M_2 = np.power(challenge.Windgeschwindigkeit152M, 2)
challenge.insert(8, 'windspeed_avg', ch_windspeed_avg)
challenge.insert(9, 'windspeed_sum', ch_windspeed_sum)
challenge.insert(10, 'Windgeschwindigkeit48M_2', ch_Windgeschwindigkeit48M_2)
challenge.insert(11, 'Windgeschwindigkeit100M_2', ch_Windgeschwindigkeit100M_2)
challenge.insert(12, 'Windgeschwindigkeit152M_2', ch_Windgeschwindigkeit152M_2)
challenge = challenge.drop(['Windgeschwindigkeit100MP10','Windgeschwindigkeit100MP20','Windgeschwindigkeit100MP30','Windgeschwindigkeit100MP40','Windgeschwindigkeit100MP50','Windgeschwindigkeit100MP60','Windgeschwindigkeit100MP70','Windgeschwindigkeit100MP80','Windgeschwindigkeit100MP90'], axis = 1)

In [15]:
data.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Min', 'Windgeschwindigkeit48M',
       'Windgeschwindigkeit100M', 'Windgeschwindigkeit152M', 'windspeed_avg',
       'windspeed_sum', 'Windgeschwindigkeit48M_2',
       'Windgeschwindigkeit100M_2', 'Windgeschwindigkeit152M_2',
       'Windrichtung48M', 'Windrichtung100M', 'Windrichtung152M',
       'Interpoliert', 'Verfügbare_Kapazität', 'Output'],
      dtype='object')

In [20]:
#4 Random Forest
from sklearn.ensemble import RandomForestRegressor

def Randomforest_original(X_train, y_train, X_test, depth):
    rf_regr = RandomForestRegressor(max_depth=depth, random_state=rng, n_jobs=2)
    rf_regr.fit(X_train, y_train)
    y_pred = rf_regr.predict(X_test)
    return y_pred

In [16]:
interval = [1, 2, 4, 6, 8, 12,16, 24, 48, 96,144, 288, 400, 500, 672,960,1024, 1440, 1960, 2880, 4096, 8640, 12960, 17280]

16384

In [15]:
#interval = []
#for i in range(1,130):
#    interval.append(i**2)

In [16]:
interval_min = []
for i in interval:
    interval_min.append(int(i*15))

In [17]:
g = globals()
variables = []
for i, j in zip(interval, interval_min):
    g["T_M{}".format(j)] = data.Output.shift(i)
    variables.append("T_M{}".format(j))

In [18]:
tvc = 52507
rng = np.random.RandomState(2028)
for i,j,k,l in zip(interval, interval_min, variables, range(16,16+len(variables))):
    X = scale(np.array(data.loc[:len(data),'Hour':'Interpoliert']))
    X = X.astype(np.float32)
    offset = int(X.shape[0] * 0.66)
    X_train= X[:offset]
    data.insert(l, "T_M{}".format(j), eval(k))
    X_train,y_train = X[i:i+tvc], getattr(data, "T_M%d"%j)[i:i+tvc]
    X_test, y_test = X[i+tvc:], getattr(data, "T_M%d"%j)[i+tvc:]
    X_test2, y_test2 = X[:i], getattr(data, "T_M%d"%j)[:i]
    getattr(data, "T_M%d"%j)[i+tvc:] = Randomforest_original(X_train, y_train, X_test, 9)
    getattr(data, "T_M%d"%j)[:i] = Randomforest_original(X_train, y_train, X_test2, 9)
    print(k, "added")

/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


T_M15 added


/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


T_M30 added


/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


...

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


T_M194400 added


/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


T_M259200 added


/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [19]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWar

In [20]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [21]:
#1 Random Forest CV
from sklearn.ensemble import RandomForestRegressor

def Randomforest_cv(Xtrain, ytrain, Xtest, ytest, ycapacity, depth):
    rf_regr = RandomForestRegressor(max_depth=depth, random_state=rng)
    rf_regr.fit(Xtrain, ytrain)
    y_pred = rf_regr.predict(Xtest)
    cape = sum(abs(y_pred - ytest))/sum(ycapacity)
    print("Random Forest CAPE", cape)
    return y_pred

In [22]:
rf_feature = []
for i, j in kf.split(X):
    rf_pred = Randomforest_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j], 7)
    print(i,j)
    for k in rf_pred:
        rf_feature.append(k)

Random Forest CAPE 0.03615443479694251
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
Random Forest CAPE 0.027414217145302504
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
Random Forest CAPE 0.025610025966184682
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
Random Forest CAPE 0.02604510402669217
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
Random Forest CAPE 0.026720058368747933
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [23]:
def Randomforest_ch(Xtrain, ytrain, Xtest, depth):
    rf_regr = RandomForestRegressor(max_depth=depth, random_state=rng)
    rf_regr.fit(Xtrain, ytrain)
    y_pred = rf_regr.predict(Xtest)
    return list(y_pred)

In [24]:
rf_challenge = Randomforest_ch(X, y, X_test_challenge, 9)

In [25]:
data.insert(16, 'RandomForest', rf_feature + rf_challenge)

In [26]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [27]:
#2 Linear Regression CV

from sklearn import datasets, linear_model
def LinearRegression_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    regr = linear_model.LinearRegression()
    regr.fit(Xtrain, ytrain)
    y_pred = regr.predict(Xtest)    
    cape = sum(abs(y_pred - ytest))/sum(ycapacity)
    print("Linear Regression CAPE /  Original Training Data / 10% Test data: ", cape)
    return y_pred

In [28]:
lr_feature = []
for i, j in kf.split(X):
    lr_pred = LinearRegression_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in lr_pred:
        lr_feature.append(k)

Linear Regression CAPE /  Original Training Data / 10% Test data:  0.03430830541997546
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
Linear Regression CAPE /  Original Training Data / 10% Test data:  0.02721329957135947
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
Linear Regression CAPE /  Original Training Data / 10% Test data:  0.0253889233514817
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
Linear Regression CAPE /  Original Training Data / 10% Test data:  0.026340918124590766
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
Linear Regression CAPE /  Original Training Data / 10% Test data:  0.02674313023092734
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [29]:
def LinearRegression_ch(Xtrain, ytrain, Xtest):
    lr_regr = linear_model.LinearRegression()
    lr_regr.fit(Xtrain, ytrain)
    y_pred = lr_regr.predict(Xtest)
    return list(y_pred)

In [30]:
lr_challenge = LinearRegression_ch(X, y, X_test_challenge)

In [31]:
data.insert(17, 'Linear', lr_feature + lr_challenge)

In [32]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [33]:
data.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Min', 'Windgeschwindigkeit48M',
       'Windgeschwindigkeit100M', 'Windgeschwindigkeit152M', 'windspeed_avg',
       'windspeed_sum', 'Windgeschwindigkeit48M_2',
       'Windgeschwindigkeit100M_2', 'Windgeschwindigkeit152M_2',
       'Windrichtung48M', 'Windrichtung100M', 'Windrichtung152M',
       'RandomForest', 'Linear', 'T_M15', 'T_M30', 'T_M60', 'T_M90', 'T_M120',
       'T_M180', 'T_M240', 'T_M360', 'T_M720', 'T_M1440', 'T_M2160', 'T_M4320',
       'T_M6000', 'T_M7500', 'T_M10080', 'T_M14400', 'T_M15360', 'T_M21600',
       'T_M29400', 'T_M43200', 'T_M61440', 'T_M129600', 'T_M194400',
       'T_M259200', 'Interpoliert', 'Verfügbare_Kapazität', 'Output'],
      dtype='object')

In [34]:
#3 Decision Tree Regression
from sklearn.tree import DecisionTreeRegressor
def Decisiontree_cv(Xtrain, ytrain, Xtest, ytest, ycapacity, depth):
    regr = DecisionTreeRegressor(max_depth=depth)
    regr.fit(Xtrain, ytrain)
    y_pred = regr.predict(Xtest)
    cape = sum(abs(y_pred - ytest))/sum(ycapacity)
    print("Decision Tree / 5 fold CV ", cape)
    return y_pred

In [35]:
dt_feature = []
for i, j in kf.split(X):
    dt_pred = Decisiontree_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j], 7)
    print(i,j)
    for k in dt_pred:
        dt_feature.append(k)

Decision Tree / 5 fold CV  0.03633095605416804
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
Decision Tree / 5 fold CV  0.027804447346894967
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
Decision Tree / 5 fold CV  0.025466115056470666
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
Decision Tree / 5 fold CV  0.027714764203930216
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
Decision Tree / 5 fold CV  0.026906543013336007
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [36]:
def Decisiontree_ch(Xtrain, ytrain, Xtest, depth):
    dt_regr = DecisionTreeRegressor(max_depth=depth)
    dt_regr.fit(Xtrain, ytrain)
    y_pred = dt_regr.predict(Xtest)
    return list(y_pred)

In [37]:
dt_challenge = Decisiontree_ch(X, y, X_test_challenge, 7)
data.insert(18, 'DecisionTree', dt_feature + dt_challenge)

In [38]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [39]:
#4 Bagging regression
from sklearn.ensemble import BaggingRegressor
def bagging_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    bag = BaggingRegressor(n_estimators=5,random_state=rng,n_jobs=2)
    y_pred_bag = bag.fit(Xtrain, ytrain).predict(Xtest)
    cape = sum(abs(y_pred_bag - ytest))/sum(ycapacity)
    print("Bagging CAPE /  Original Training Data / 10% Test data: ", cape)
    return y_pred_bag

In [40]:
bg_feature = []
for i, j in kf.split(X):
    bg_pred = bagging_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in bg_pred:
        bg_feature.append(k)

Bagging CAPE /  Original Training Data / 10% Test data:  0.04034626435968392
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
Bagging CAPE /  Original Training Data / 10% Test data:  0.030683998190229676
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
Bagging CAPE /  Original Training Data / 10% Test data:  0.027949829209734232
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
Bagging CAPE /  Original Training Data / 10% Test data:  0.03377557320242108
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
Bagging CAPE /  Original Training Data / 10% Test data:  0.03006581149627064
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [41]:
def bagging_ch(Xtrain, ytrain, Xtest):
    bag = BaggingRegressor(n_estimators=5,random_state=rng,n_jobs=2)
    y_pred_bag = bag.fit(Xtrain, ytrain).predict(Xtest)
    return list(y_pred_bag)

In [42]:
bg_challenge = bagging_ch(X, y, X_test_challenge)
data.insert(19, 'Bagging', bg_feature + bg_challenge)

In [43]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [44]:
#5 MLP Regressor
from sklearn.neural_network import MLPRegressor
def mlp_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    reg = MLPRegressor(hidden_layer_sizes=(10,),  activation='relu', solver='adam',    alpha=0.01,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.05, power_t=0.5, max_iter=1000, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)

    mlp = reg.fit(Xtrain, ytrain)
    y_pred = mlp.predict(Xtest)
    try:
        cape = sum(abs(y_pred - ytest))/sum(ycapacity)
        print("CNN CAPE /  Original Training Data / 10% Test data: ", cape)
    except:
        pass
    return y_pred

In [45]:
mlp_feature = []
for i, j in kf.split(X):
    mlp_pred = mlp_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in mlp_pred:
         mlp_feature.append(k)

CNN CAPE /  Original Training Data / 10% Test data:  0.03728438603012265
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
CNN CAPE /  Original Training Data / 10% Test data:  0.02738091732107605
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
CNN CAPE /  Original Training Data / 10% Test data:  0.02586152193086152
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
CNN CAPE /  Original Training Data / 10% Test data:  0.027961062545496487
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
CNN CAPE /  Original Training Data / 10% Test data:  0.026964876911430766
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [46]:
def mlp_ch(Xtrain, ytrain, Xtest):
    reg = MLPRegressor(hidden_layer_sizes=(10,),  activation='relu', solver='adam',    alpha=0.01,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.05, power_t=0.5, max_iter=1000, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)
    mlp = reg.fit(Xtrain, ytrain)
    y_pred = mlp.predict(Xtest)
    return list(y_pred)

In [47]:
mlp_challenge = mlp_ch(X, y, X_test_challenge)
data.insert(20, 'MLP', mlp_feature + mlp_challenge)

In [48]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [49]:
#6 Ridge Regrssion
from sklearn.linear_model import Ridge
def ridge_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    clf = Ridge(alpha=1.0)
    clf.fit(Xtrain, ytrain)
    y_ridge_pred = clf.predict(Xtest)
    cape = sum(abs(y_ridge_pred - ytest))/sum(ycapacity)
    print("Ridge CAPE /  Original Training Data / 10% Test data: ", cape)
    return y_ridge_pred

In [50]:
ridge_feature = []
for i, j in kf.split(X):
    ridge_pred = ridge_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in ridge_pred:
         ridge_feature.append(k)

Ridge CAPE /  Original Training Data / 10% Test data:  0.034672688482883844
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
Ridge CAPE /  Original Training Data / 10% Test data:  0.026870846981680956
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
Ridge CAPE /  Original Training Data / 10% Test data:  0.025051397930423037
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
Ridge CAPE /  Original Training Data / 10% Test data:  0.025924727936308185
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
Ridge CAPE /  Original Training Data / 10% Test data:  0.026136411923600094
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [51]:
def ridge_ch(Xtrain, ytrain, Xtest):
    clf = Ridge(alpha=1.0)
    clf.fit(Xtrain, ytrain)
    y_ridge_pred = clf.predict(Xtest)
    return list(y_ridge_pred)

In [52]:
ridge_challenge = ridge_ch(X, y, X_test_challenge)
data.insert(21, 'Ridge', ridge_feature + ridge_challenge)

In [53]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [54]:
#7 Light GBM CV
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.01, n_estimators=1000,
                              max_bin = 30, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
def lgb_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    model_lgb.fit(Xtrain, ytrain)
    lgb_pred = model_lgb.predict(Xtest)
    cape = sum(abs(lgb_pred - ytest))/sum(ycapacity)
    print("LightGBM CAPE /  Training Data 90% / 10% Test data: ", cape)
    return lgb_pred

In [55]:
lgb_feature = []
for i, j in kf.split(X):
    lgb_pred = lgb_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in lgb_pred:
         lgb_feature.append(k)

LightGBM CAPE /  Training Data 90% / 10% Test data:  0.038831468370932626
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
LightGBM CAPE /  Training Data 90% / 10% Test data:  0.027769126047329905
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
LightGBM CAPE /  Training Data 90% / 10% Test data:  0.025956361160027194
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
LightGBM CAPE /  Training Data 90% / 10% Test data:  0.028998705699038542
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
LightGBM CAPE /  Training Data 90% / 10% Test data:  0.027153381133527704
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [56]:
def lgb_ch(Xtrain, ytrain, Xtest):
    model_lgb.fit(Xtrain, ytrain)
    lgb_pred = model_lgb.predict(Xtest)
    return list(lgb_pred)

In [57]:
lgb_challenge = lgb_ch(X, y, X_test_challenge)
data.insert(22, 'LGB', lgb_feature + lgb_challenge)

In [58]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [59]:
#8 XGboost
import xgboost as xgb
def xgb_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    gbm = xgb.XGBRegressor(max_depth=4, n_estimators=500, learning_rate=0.01).fit(Xtrain, ytrain)
    predictions = gbm.predict(Xtest)
    cape = sum(abs(predictions - ytest))/sum(ycapacity)
    print("XGBoost CAPE /  Original Training Data / 10% Test data: ", cape)
    return predictions

In [60]:
xgb_feature = []
for i, j in kf.split(X):
    xgb_pred = xgb_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in xgb_pred:
         xgb_feature.append(k)

XGBoost CAPE /  Original Training Data / 10% Test data:  0.03579052125043428
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
XGBoost CAPE /  Original Training Data / 10% Test data:  0.027221050762672346
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
XGBoost CAPE /  Original Training Data / 10% Test data:  0.02503978015280855
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
XGBoost CAPE /  Original Training Data / 10% Test data:  0.027178839254272672
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
XGBoost CAPE /  Original Training Data / 10% Test data:  0.026198648633565563
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [61]:
def xgb_ch(Xtrain, ytrain, Xtest):
    gbm = xgb.XGBRegressor(max_depth=4, n_estimators=500, learning_rate=0.01).fit(Xtrain, ytrain)
    predictions = gbm.predict(Xtest)
    return list(predictions)

In [62]:
xgb_challenge = xgb_ch(X, y, X_test_challenge)
data.insert(23, 'XGB', xgb_feature + xgb_challenge)

In [63]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [64]:
#9 ElasticNet
from sklearn.linear_model import ElasticNet
def enet_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    alpha = 0.63
    enet = ElasticNet(alpha=alpha, l1_ratio=0.35)
    y_pred_enet = enet.fit(Xtrain, ytrain).predict(Xtest)
    cape = sum(abs(y_pred_enet - ytest))/sum(ycapacity)
    print("ElasticNet CAPE /  Original Training Data / 10% Test data: ", cape)
    return y_pred_enet

In [65]:
enet_feature = []
for i, j in kf.split(X):
    enet_pred = enet_cv(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j])
    print(i,j)
    for k in enet_pred:
         enet_feature.append(k)

ElasticNet CAPE /  Original Training Data / 10% Test data:  0.03988317060912024
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
ElasticNet CAPE /  Original Training Data / 10% Test data:  0.030684938912320985
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
ElasticNet CAPE /  Original Training Data / 10% Test data:  0.029257475282886244
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
ElasticNet CAPE /  Original Training Data / 10% Test data:  0.03368641801021693
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
ElasticNet CAPE /  Original Training Data / 10% Test data:  0.031149257398700607
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [66]:
def enet_ch(Xtrain, ytrain, Xtest):
    alpha = 0.63
    enet = ElasticNet(alpha=alpha, l1_ratio=0.35)
    y_pred_enet = enet.fit(Xtrain, ytrain).predict(Xtest)
    return list(y_pred_enet)

In [67]:
enet_challenge = enet_ch(X, y, X_test_challenge)
# data.insert(24, 'Enet', enet_feature + enet_challenge)

In [68]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [69]:
#9 Random Forest 2 CV
from sklearn.ensemble import RandomForestRegressor

def Randomforest_cv2(Xtrain, ytrain, Xtest, ytest, ycapacity, depth):
    rf_regr = RandomForestRegressor(max_depth=depth, random_state=rng)
    rf_regr.fit(Xtrain, ytrain)
    y_pred = rf_regr.predict(Xtest)
    cape = sum(abs(y_pred - ytest))/sum(ycapacity)
    print("Random Forest CAPE", cape)
    return y_pred

In [70]:
rf_feature2 = []
for i, j in kf.split(X):
    rf_pred = Randomforest_cv2(X[i], y[i], X[j], y[j], data.Verfügbare_Kapazität[j], 9)
    print(i,j)
    for k in rf_pred:
        rf_feature2.append(k)

Random Forest CAPE 0.0369759934396712
[10502 10503 10504 ... 52505 52506 52507] [    0     1     2 ... 10499 10500 10501]
Random Forest CAPE 0.027451473056381393
[    0     1     2 ... 52505 52506 52507] [10502 10503 10504 ... 21001 21002 21003]
Random Forest CAPE 0.024923240777336854
[    0     1     2 ... 52505 52506 52507] [21004 21005 21006 ... 31503 31504 31505]
Random Forest CAPE 0.029076042228365444
[    0     1     2 ... 52505 52506 52507] [31506 31507 31508 ... 42004 42005 42006]
Random Forest CAPE 0.02651507204162057
[    0     1     2 ... 42004 42005 42006] [42007 42008 42009 ... 52505 52506 52507]


In [71]:
def Randomforest_ch2(Xtrain, ytrain, Xtest, depth):
    rf_regr = RandomForestRegressor(max_depth=depth, random_state=rng)
    rf_regr.fit(Xtrain, ytrain)
    y_pred = rf_regr.predict(Xtest)
    return list(y_pred)

In [72]:
rf_challenge2 = Randomforest_ch2(X, y, X_test_challenge, 9)

In [73]:
data.insert(24, 'RandomForest2', rf_feature2 + rf_challenge2)

In [74]:
#Data Split & Scaling
X = scale(np.array(data.ix[:52507,'Hour':'Interpoliert']))
y = np.array(data.Output.ix[:52507])
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
X_test_challenge = scale(np.array(data.ix[52508:,'Hour':'Interpoliert']))
y_test_challenge = np.array(data.ix[52508:].Output)
y_capacity = np.array(data.Verfügbare_Kapazität.ix[offset:52507])

/home/shin/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [75]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.03, n_estimators=1000,
                              max_bin = 30, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
def lightgbm(Xtrain, ytrain, Xtest, ytest, ycapacity):
    model_lgb.fit(Xtrain, ytrain)
    lgb_pred = model_lgb.predict(Xtest)
    cape = sum(abs(lgb_pred - ytest))/sum(ycapacity)
    print("LightGBM CAPE /  Training Data 90% / 10% Test data: ", cape)
    return lgb_pred

In [76]:
lightgbm(X_train, y_train, X_test, y_test, y_capacity)

LightGBM CAPE /  Training Data 90% / 10% Test data:  0.0244407730578272


array([ 7262.05804327,  8299.31304279,  7800.20258187, ...,
       17290.71235591, 17543.4277966 , 14862.77566938])

In [77]:
lightgbm(X_train, y_train, X_test, y_test, y_capacity)

LightGBM CAPE /  Training Data 90% / 10% Test data:  0.0244407730578272


array([ 7262.05804327,  8299.31304279,  7800.20258187, ...,
       17290.71235591, 17543.4277966 , 14862.77566938])

In [112]:
def xgb_cv(Xtrain, ytrain, Xtest, ytest, ycapacity):
    gbm = xgb.XGBRegressor(max_depth=7, n_estimators=1000, learning_rate=0.01).fit(Xtrain, ytrain)
    predictions = gbm.predict(Xtest)
    cape = sum(abs(predictions - ytest))/sum(ycapacity)
    print("XGBoost CAPE /  Original Training Data / 10% Test data: ", cape)
    return predictions
xgb_cv(X_train, y_train, X_test, y_test, y_capacity)

XGBoost CAPE /  Original Training Data / 10% Test data:  0.023967201870621468


array([ 7714.0776,  8915.644 ,  8292.856 , ..., 18210.139 , 18690.375 ,
       15037.096 ], dtype=float32)

In [79]:
Randomforest_cv(X_train, y_train, X_test, y_test, y_capacity, 9)

Random Forest CAPE 0.024376032519876877


array([ 7815.05474824,  9061.73323667,  8428.67375235, ...,
       17540.04548426, 18183.72102714, 14904.44434258])

In [80]:
#lightgbm 0.024413401288697217
#Random Forest 0.02407022221515361
#MLP 0.024730823589527717
#XGB  0.024175183128886082 depth =4, n=500 lr = 0.01
#XGB  0.02404250686442158 depth =4, n=1000 lr = 0.01
#XGB 0.02405902502124753 depth =4, n= 1000 lr = 0.03
#XGB 0.024376032519876877 depth =4, n= 1300 lr = 0.01
#XGB 0.02403249765675427 depth =5, n= 1000 lr = 0.03
#XGB 0.02399690647455083 max_depth=6, n_estimators=1000, learning_rate=0.01
#XGB 0.0240423412097441 max_depth=6, n_estimators=1000, learning_rate=0.005
#XGB 0.023967201870621468 max_depth=7, n_estimators=1000, learning_rate=0.01
#XGB  0.024362709992839744 max_depth=9, n_estimators=1000, learning_rate=0.01

In [81]:
mlp_cv(X_train, y_train, X_test, y_test, y_capacity)

CNN CAPE /  Original Training Data / 10% Test data:  0.02459076294986711


array([ 7252.97404195,  8263.776241  ,  7854.68156657, ...,
       18168.1848841 , 18526.32180454, 15778.64069757])

In [82]:
def lightgbm_challenge(Xtrain, ytrain, Xtest):
    model_lgb.fit(Xtrain, ytrain)
    lgb_pred = model_lgb.predict(Xtest)
    return lgb_pred

In [113]:
def xgb_challenge(Xtrain, ytrain, Xtest):
    gbm = xgb.XGBRegressor(max_depth=7, n_estimators=1000, learning_rate=0.005).fit(Xtrain, ytrain)
    predictions = gbm.predict(Xtest)
    return predictions

In [114]:
submission = xgb_challenge(X, y, X_test_challenge)

KeyboardInterrupt: 

In [ ]:
challenge2 = pd.read_csv('/home/shin/Documents/Data_mining/challenge.csv', sep = ',')

In [ ]:
challenge2.Output = submission

In [ ]:
challenge2.to_csv("submission8-1.csv", sep=',', index=False)

In [87]:
data[:52508].to_csv("training_stacking.csv", sep=',', index=False)

In [88]:
data[52508:].to_csv("challenge_stacking.csv", sep=',', index=False, header = True)

In [89]:
data.to_csv("alldata.csv", sep=',', index=False, header=False)